# Quotes for swaps

In [ ]:
#| default_exp quote

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from functools import reduce
from typing import List, Union
from eth_abi.packed import encode_packed
from dataclasses import dataclass
from sugar.token import Token
from sugar.pool import LiquidityPoolForSwap

In [ ]:
#| export 

@dataclass
class Quote: from_token: Token; to_token: Token; path: List[LiquidityPoolForSwap]; amount_in: int; amount_out: int

# magic numbers
QUOTER_STABLE_POOL_FILLER, QUOTER_VOLATILE_POOL_FILLER = 2097152, 4194304

@dataclass
class PreparedRoute: 
    types: List[str]; values: List[Union[str, int]]

    @property
    def encoded(self) -> bytes: return encode_packed(self.types, self.values)


def prepare_route(nodes: List[LiquidityPoolForSwap]) -> PreparedRoute:
    types, values = reduce(lambda s, pool: s + pool, [["address", "int24"] for i in range(len(nodes))], []) + ["address"], []
    for pool in nodes:
        if pool.type > 0: filler = pool.type
        else: filler =  QUOTER_STABLE_POOL_FILLER if pool.is_stable else QUOTER_VOLATILE_POOL_FILLER
        if len(values) == 0: values = [pool.token0.token_address, filler, pool.token1.token_address]
        else: values += [filler, pool.token1.token_address]
    return PreparedRoute(types=types, values=values)

In [ ]:
from fastcore.test import test_eq

nodes = [
    LiquidityPoolForSwap(lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=200, token0=Token(token_address='0x4200000000000000000000000000000000000006', symbol='WETH', decimals=18, listed=True, wrapped_token_address=None), token1=Token(token_address='0x6c84a8f1c29108F47a79964b5Fe888D4f4D0dE40', symbol='tBTC', decimals=18, listed=True, wrapped_token_address=None), is_stable=False, is_cl=True, symbol='CL200-WETH/tBTC'),
    LiquidityPoolForSwap(lp='0x02A130b6D35611bC2D90e20f2ceA45431c0A9a8d', type=-1, token0=Token(token_address='0x6c84a8f1c29108F47a79964b5Fe888D4f4D0dE40', symbol='tBTC', decimals=18, listed=True, wrapped_token_address=None), token1=Token(token_address='0x7F5c764cBc14f9669B88837ca1490cCa17c31607', symbol='USDC', decimals=6, listed=True, wrapped_token_address=None), is_stable=False, is_cl=False, symbol='vAMM-tBTC/USDC'),
    LiquidityPoolForSwap(lp='0xf7a73e1995030B0198f8d6e528a1c42ACEf90a4c', type=0, token0=Token(token_address='0x7F5c764cBc14f9669B88837ca1490cCa17c31607', symbol='USDC', decimals=6, listed=True, wrapped_token_address=None), token1=Token(token_address='0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', symbol='VELO', decimals=18, listed=True, wrapped_token_address=None), is_stable=True, is_cl=False, symbol='sAMM-USDC/VELO')
]

rt = prepare_route(nodes)

test_eq(rt.types, ['address', 'int24', 'address', 'int24', 'address', 'int24', 'address'])
test_eq(rt.values, ['0x4200000000000000000000000000000000000006', 200, '0x6c84a8f1c29108F47a79964b5Fe888D4f4D0dE40', 4194304, '0x7F5c764cBc14f9669B88837ca1490cCa17c31607', 2097152, '0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db'])
test_eq(rt.encoded.hex(), "42000000000000000000000000000000000000060000c86c84a8f1c29108f47a79964b5fe888d4f4d0de404000007f5c764cbc14f9669b88837ca1490cca17c316072000009560e827af36c94d2ac33a39bce1fe78631088db")    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()